In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
## Read data from file
gen_table = pd.read_csv('gen_table.csv')
gen_table

,Amino Acid,Fitness
0,GREEDQNSCHTKTFLLIFPVAALYCAQCMAGCFCNLRLYGTYRGYD...,27
1,RREEDEDHQKGHGTSALPSLALSHHVALASGERAHPIVHHPSIPTQ...,27
2,RRRDDSQSNSSTAVWNGIVAFCFRFDRIGRHSRFVNVILNLRPAGE...,20
3,RIEEHGDHQKGHAKRSGLIIQPISGLLNSSLLPPYANFDLERLRRE...,24
4,RRRDDSDCSNGSTMVAGAVDILLGSGVVELLSVKNLGTEPAVRREK...,26
5,RRRYDSDTCSNSSTTVLGAVLNGDSIRVGPVWAVALPTVNAFTAEK...,31
6,RRRDDSDCSNGSTRFGVARPLCYNGTAVDRIGVYLSGRHHANGKVP...,25
7,RRRDDSDCSNGSTTFPRPKILAISVTMVPVCLPGNTVIRIVAAGQK...,28
8,RRRDDSDCSNGSTTILQIFGVARLDKAPFDRHLSLHLIHTLRRVKE...,15
9,RREEDEDHQNSGSTNGGVLPLLVKCCCVLYGLLLFLIIIQFMRREK...,39


In [39]:
gen_array = gen_table["Amino Acid"]
gen_fit = gen_table["Fitness"]
gen_array

0    GREEDQNSCHTKTFLLIFPVAALYCAQCMAGCFCNLRLYGTYRGYD...
1    RREEDEDHQKGHGTSALPSLALSHHVALASGERAHPIVHHPSIPTQ...
2    RRRDDSQSNSSTAVWNGIVAFCFRFDRIGRHSRFVNVILNLRPAGE...
3    RIEEHGDHQKGHAKRSGLIIQPISGLLNSSLLPPYANFDLERLRRE...
4    RRRDDSDCSNGSTMVAGAVDILLGSGVVELLSVKNLGTEPAVRREK...
5    RRRYDSDTCSNSSTTVLGAVLNGDSIRVGPVWAVALPTVNAFTAEK...
6    RRRDDSDCSNGSTRFGVARPLCYNGTAVDRIGVYLSGRHHANGKVP...
7    RRRDDSDCSNGSTTFPRPKILAISVTMVPVCLPGNTVIRIVAAGQK...
8    RRRDDSDCSNGSTTILQIFGVARLDKAPFDRHLSLHLIHTLRRVKE...
9    RREEDEDHQNSGSTNGGVLPLLVKCCCVLYGLLLFLIIIQFMRREK...
Name: Amino Acid, dtype: object

In [40]:
gen_fit

0    27
1    27
2    20
3    24
4    26
5    31
6    25
7    28
8    15
9    39
Name: Fitness, dtype: int64

In [ ]:
## 

In [ ]:
# pop_array = []
# with open("pop_array.csv", "r") as popfile: 
#     reader = csv.reader(popfile)
#     for row in reader:
#         pop_array.append(''.join(row))

# pop_array

In [ ]:
amino_acids = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
    'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19
}

blosum50_matrix = [
    [ 5, -2, -1, -2, -1, -1, -1,  0, -2, -1, -2, -1, -1, -3, -1,  1,  0, -3, -2,  0],
    [-2,  7, -1, -2, -4,  1,  0, -3,  0, -4, -3,  3, -2, -3, -3, -1, -1, -3, -1, -3],
    [-1, -1,  7,  2, -2,  0,  0,  0,  1, -3, -4,  0, -2, -4, -2,  1,  0, -4, -2, -3],
    [-2, -2,  2,  8, -4,  0,  2, -1, -1, -4, -4, -1, -4, -5, -1,  0, -1, -5, -3, -4],
    [-1, -4, -2, -4, 13, -3, -3, -3, -3, -2, -2, -3, -2, -2, -4, -1, -1, -5, -3, -1],
    [-1,  1,  0,  0, -3,  7,  2, -2,  1, -3, -2,  2,  0, -4, -1,  0, -1, -1, -1, -3],
    [-1,  0,  0,  2, -3,  2,  6, -3,  0, -4, -3,  1, -2, -3, -1,  -1, -1, -3, -2, -3],
    [ 0, -3,  0, -1, -3, -2, -3,  8, -2, -4, -4, -2, -3, -4, -2,  0, -2, -3, -3, -4],
    [-2,  0,  1, -1, -3,  1,  0, -2, 10, -4, -3,  0, -1, -1, -2, -1, -2, -3,  2, -4],
    [-1, -4, -3, -4, -2, -3, -4, -4, -4,  5,  2, -3,  2,  0, -3, -3, -1, -3, -1,  4],
    [-2, -3, -4, -4, -2, -2, -3, -4, -3,  2,  5, -3,  3,  1, -4, -3, -1, -2, -1,  1],
    [-1,  3,  0, -1, -3,  2,  1, -2,  0, -3, -3,  6, -2, -4, -1,  0, -1, -3, -2, -3],
    [-1, -2, -2, -4, -2,  0, -2, -3, -1,  2,  3, -2,  7,  0, -3, -2, -1, -1,  0,  1],
    [-3, -3, -4, -5, -2, -4, -3, -4, -1,  0,  1, -4,  0,  8, -4, -3, -2,  1,  4, -1],
    [-1, -3, -2, -1, -4, -1, -1, -2, -2, -3, -4, -1, -3, -4, 10, -1, -1, -4, -3, -3],
    [ 1, -1,  1,  0, -1,  0,  -1,  0, -1, -3, -3,  0, -2, -3, -1,  5,  2, -4, -2, -2],
    [ 0, -1,  0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1,  2,  5, -3, -2,  0],
    [-3, -3, -4, -5, -5, -1, -3, -3, -3, -3, -2, -3, -1,  1, -4, -4, -3, 15,  2, -3],
    [-2, -1, -2, -3, -3, -1, -2, -3,  2, -1, -1, -2, 0,  4, -3, -2, -2,  2,  8, -1],
    [ 0, -3, -3, -4, -1, -3, -3, -4, -4,  4,  1, -3,  1, -1, -3, -2,  0, -3, -1,  5]
]

In [ ]:
class align:
    
    def __init__(self, str1, str2):
        self.str1 = str1
        self.str2 = str2
        self.gapP = -8
        self.score = [[0 for x in range(0,len(self.str1)+1)] for y in range(0,len(self.str2)+1)]
        self.direction = [[0 for x in range(0,len(self.str1)+1)] for y in range(0,len(self.str2)+1)]

    def my_print(self,array):
        for r in array:
            for c in r:
                print(f"{c:4}",end = ' ')
            print()

    def fill_matrix(self):
        for c in range(0,len(self.str1)+1):
            self.score[0][c] = c*self.gapP
            self.direction[0][c] = '-'
        for r in range(0,len(self.str2)+1):
            self.score[r][0] = r*self.gapP
            self.direction[r][0] = '|'
        self.direction[0][0] = 'X'
        # self.my_print(self.score)
        # self.my_print(self.direction)

    # def SW(self):
    #     for c in range(1,len(self.str1)+1):
    #         for r in range(1,len(self.str2)+1):
    #             scoreD = self.score[r-1][c-1] + blosum50_matrix[amino_acids[self.str2[r-1]]][amino_acids[self.str1[c-1]]]
    #             scoreV = self.score[r-1][c] + self.gapP 
    #             scoreH = self.score[r][c-1] + self.gapP
    #             # set to initial index in columns thats always 0
    #             scoreZ = 0
    #             self.score[r][c] = max(scoreD,scoreV,scoreH,scoreZ)
    #             if(self.score[r][c] == scoreD):
    #                 self.direction[r][c] = "\\"
    #             if(self.score[r][c] == scoreH):
    #                 self.direction[r][c] = "-"
    #             if(self.score[r][c] == scoreV):
    #                 self.direction[r][c] = "|"

    #     self.my_print(self.score)
    #     self.my_print(self.direction)

    def NW(self):
        for c in range(1,len(self.str1)+1):
            for r in range(1,len(self.str2)+1):
                scoreD = self.score[r-1][c-1] + blosum50_matrix[amino_acids[self.str2[r-1]]][amino_acids[self.str1[c-1]]]
                scoreV = self.score[r-1][c] + self.gapP
                scoreH = self.score[r][c-1] + self.gapP
                self.score[r][c] = max(scoreD,scoreV,scoreH)
                if(self.score[r][c] == scoreD):
                    self.direction[r][c] = "\\"
                if(self.score[r][c] == scoreH):
                    self.direction[r][c] = "-"
                if(self.score[r][c] == scoreV):
                    self.direction[r][c] = "|"
        
    #     self.my_print(self.score)
    #     self.my_print(self.direction)


    def local_alignment(self):
        align1 = ""
        align2 = ""

        # For LOCAL ALIGNMENT
        target = max(map(max, self.score))
        print("Largest value in the matrix is: ", target)
        s = [start for start in self.score if target in start][0]

        print ('The index is (%d,%d)'%(self.score.index(s), s.index(target)))
        
        row = self.score.index(s)
        col = s.index(target)


        while(row > 0 or col > 0):
            if(self.direction[row][col] == "\\"):
                align1 += self.str1[col-1]
                align2 += self.str2[row-1]
                row -= 1
                col -= 1
            if(self.direction[row][col] == "-"):
                align1 += self.str1[col-1]
                align2 += "-"
                col -= 1
            if(self.direction[row][col] == "|"):
                align1 += "-"
                align2 += self.str2[row-1]
                row -= 1
            if(self.direction[row][col] == 0):
                break
        align1 = align1[::-1]
        align2 = align2[::-1]

        print(align1)
        print(align2)
        return align1, align2, target
    
    def global_alignment(self):
        align1 = ""
        align2 = ""
        row = len(self.str2)
        col = len(self.str1)
        while(row > 0 or col > 0):
            if(self.direction[row][col] == "\\"):
                align1 += self.str1[col-1]
                align2 += self.str2[row-1]
                row -= 1
                col -= 1
            if(self.direction[row][col] == "-"):
                align1 += self.str1[col-1]
                align2 += "-"
                col -= 1
            if(self.direction[row][col] == "|"):
                align1 += "-"
                align2 += self.str2[row-1]
                row -= 1
        align1 = align1[::-1]
        align2 = align2[::-1]
        target = self.score[len(self.str2)][len(self.str1)]
        return align1,align2, target

In [ ]:
def my_print(array):
        for r in array:
            for c in r:
                print(f"{c:4}",end = ' ')
            print()

In [ ]:
local = align(gen_array[3],gen_array[4])
local.fill_matrix()
local.NW()

print(gen_array[3])
print(gen_array[4])

str1, str2, target = local.local_alignment()

print("Score: ", target)



In [ ]:
gen_array

In [ ]:
pairwise_score = {}

score_matrix = [[0 for x in range(0, len(gen_array))] for y in range(0, len(gen_array))]

for i in range(0, len(gen_array)):
    for j in range(i+1, len(gen_array)):
        local = align(gen_array[i],gen_array[j])
        local.fill_matrix()
        local.NW()
        str1, str2, target = local.global_alignment()
        pairwise_score[(i,j)] = target
        # score_matrix[i][j] = score_matrix[j][i] = target
        score_matrix[j][i] = target


In [ ]:
my_print(score_matrix)

In [ ]:
pairwise_score

In [ ]:
sorted_pairs = sorted(pairwise_score.items(), key=lambda x:-x[1])
sorted_pairs

In [ ]:
matrix = [[0] * (i + 1) for i in range(len(gen_array))]

for (i, j), score in pairwise_score.items():
    distance = 1 - (score / max(pairwise_score.values()))
    if i < j:
        matrix[j][i] = distance

In [ ]:
matrix

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor, DistanceMatrix
from Bio import Phylo

labels = [f"Gen_{(i+1)*10}" for i in range(len(gen_array))]
labels

In [ ]:
distance_matrix = DistanceMatrix(labels, matrix)

In [ ]:
constructor = DistanceTreeConstructor()
tree = constructor.nj(distance_matrix)
Phylo.draw(tree)
plt.figure(figsize=(9,10))
plt.show()

In [ ]:
gen_array

In [ ]:
sorted_pairs[0][0][1]